In [45]:
import pandas
from ortools.linear_solver import pywraplp

# Читаем данные
# Временный вариант - читаем из экселя
# Чтение всех данных
inputData = pandas.read_excel('input.xlsx',index_col=None, header=None)
inputDataList = inputData.values.tolist()
# Срез значений обеспечений
guaranteeSum = [tmp[0] for tmp in inputDataList[1:]]
# Срез сумм кредитов
creditSum = [tmp for tmp in inputDataList[0][1:]]
# Получение матрицы соотношений кредитов и обеспечений
guaranteeMatrix = [tmp[1:] for tmp in inputDataList[1:]]

# Создаём объект для решения нашей задачи
solver = pywraplp.Solver.CreateSolver("SCIP")


# Создаём переменную - максимальное покрытие кредита обеспечением
# может принимать значение >0
max_guaranteeCoverValue = solver.NumVar(0, solver.infinity(), "")

# Создаём переменную - минимальное покрытие кредита обеспечением
# может принимать значение >0
min_guaranteeCoverValue = solver.NumVar(0, solver.infinity(), "")

# Суть задачи чтобы покрытия всех кредитов были максимально близки друг к другу
# Будем добиваться это путём минимизации значения максимум минус минимум
solver.Objective().SetMinimization()
# Устанавливаем коэффициенты для параметров
solver.Objective().SetCoefficient(max_guaranteeCoverValue, 1)
solver.Objective().SetCoefficient(min_guaranteeCoverValue, -1)


# Параметризируем матрицу соотношений кредитов и обеспечений
guaranteeMatrixValue = [[solver.NumVar(0, tmp2, "") for tmp2 in tmp] for tmp in guaranteeMatrix]

# Создаём ограничение, чтобы обеспечение было распределено между кредитами ровно на 100%
for i, guarantee in enumerate(guaranteeSum):
    distributionConstraint = solver.Constraint(1, 1)
    for j, credit in enumerate(creditSum):
        distributionConstraint.SetCoefficient(guaranteeMatrixValue[i][j], 1)

# ограничения на максимальное и минимальное покрытия
for j, credit in enumerate(creditSum):
    max_guaranteeCoverСonstraint = solver.Constraint(-solver.infinity(), 0)
    max_guaranteeCoverСonstraint.SetCoefficient(max_guaranteeCoverValue, -1)
    min_guaranteeCoverСonstraint = solver.Constraint(0, solver.infinity())
    min_guaranteeCoverСonstraint.SetCoefficient(min_guaranteeCoverValue, -1)
    for i, guarantee in enumerate(guaranteeSum):
        ratio = guarantee / credit
        max_guaranteeCoverСonstraint.SetCoefficient(guaranteeMatrixValue[i][j], ratio)
        min_guaranteeCoverСonstraint.SetCoefficient(guaranteeMatrixValue[i][j], ratio)

status = solver.Solve()

if status == solver.OPTIMAL:
    print('Solution is optimal')
else:
    if status == solver.FEASIBLE:
        print('Solution can be not optimal')     
    else:
        if status== solver.INFEASIBLE:
            print('No optimal solution for this task')
        else:
            print('Unknown status')

answer = [[solver.Objective().Value()] + creditSum]
for i, guarantee in enumerate(guaranteeSum):
    answer.append([guarantee] + [guaranteeValue.solution_value() for guaranteeValue in guaranteeMatrixValue[i]])
answer.append(['distr_val']+[sum(guarantee*answer[i+1][j+1] for i, guarantee in enumerate(guaranteeSum)) for j, credit in enumerate(creditSum)])
answer.append(['distr_%']+[answer[-1][j+1]/credit for j, credit in enumerate(creditSum)])

df = pandas.DataFrame(answer)
print(df.round(2))
df.to_excel('output.xlsx',header=False, index=False)

Solution is optimal
            0         1         2         3         4         5
0   0.0951443  68000.00  88000.00  42000.00  47000.00  29000.00
1        5000      1.00     -0.00     -0.00      0.00     -0.00
2       32000      0.26      0.64      0.10      0.00     -0.00
3       35000      0.26     -0.00      0.00      0.74     -0.00
4       11000      1.00     -0.00     -0.00      0.00     -0.00
5       14000      1.00      0.00     -0.00     -0.00     -0.00
6       23000     -0.00     -0.00     -0.00     -0.00      1.00
7        7000     -0.00      0.00      0.00      1.00     -0.00
8       34000     -0.00      1.00      0.00      0.00     -0.00
9       26000     -0.00     -0.00      1.00     -0.00     -0.00
10       7000     -0.00      1.00     -0.00      0.00     -0.00
11  distr_val  47461.22  61420.41  29314.29  32804.08  23000.00
12    distr_%      0.70      0.70      0.70      0.70      0.79
